In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import f1_score

import warnings

warnings.filterwarnings("ignore")


# Métodos de Classificação II

Vamos avançar nossos estudos sobre métodos de classificação analisando dois novos algoritmos (o *Perceptron de Múltiplas Camadas* e os *Modelos de Ensemble*). Além disso teremos o primeiro contato com os conceitos de *underfitting* e *overfitting*, e como esses conceitos se relacionam com a **complexidade** dos modelos de ML.

Primeiro, vamos carregar nosso dataset de exemplo:

In [ ]:
tb_cov = pd.read_parquet("data/covtype.parquet")
tb_cov.head()

## Conjunto de Dados de Cobertura Florestal dos EUA

Este conjunto de dados contém informações sobre a cobertura florestal de diferentes lugares dos Estados Unidos. Além da variável target, que é o tipo de árvore predominante na área, também temos informações sobre a região, como:

- Altitude
- Inclinação do terreno
- Distância até a água mais próxima

Essas informações foram obtidas a partir do processamento de imagens de satélite.

### Análise da Variável Target

A variável target é o tipo de árvore predominante na área:


In [ ]:
tb_cov["target"].value_counts()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
sns.countplot(data=tb_cov, x="target")


Como podemos ver no gráfico acima, temos um total de 7 classes de árvores. A classe predominante é a segunda no gráfico, que corresponde à árvores do tipo *Lodgepole Pine*.

Esse tipo de fenômeno é comum em problemas de classificação (como no exemplo de fraudes em cartão de crédito): **nossas classe estão desbalanceadas**. Isso significa que temos muito mais exemplos de uma classe do que de outras. No caso de fraudes, temos muito mais transações legítimas do que fraudulentas. No caso de árvores, temos muito mais *Lodgepole Pine* e *Spruce/Fir* do que o resto.

O desbalanço de classes pode ser um problema para alguns algoritmos de classificação. Por exemplo, se tivermos um algoritmo que sempre chuta a classe mais frequente, ele terá uma acurácia de 50% no nosso dataset de árvores. Isso porque 50% das árvores são *Lodgepole Pine*. Mais adiante veremos como podemos tratar esse problema. Primeiro, vamos usar um `LabelEncoder` para converter nosso target de `strings` para números de forma estruturada:



In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
le.fit(tb_cov["target"])

A utilização de um `LabelEncoder` garante que as categorias sejam sempre ordenadas da mesma forma (a primeira classe é sempre 0, a segunda é sempre 1, etc).

### Análise dos Features

In [ ]:
tb_cov.info()

Temos ao todo 12 features contendo informações sobre a localização, o solo e o clima de cada área. Entre essas, 2 são categóricas e 10 são numéricas. Vamos focar nossa análise exploratória em dois eixos: avaliar a colinearidade e distribuição de nossos variáveis numéricas.

#### Colinearidade

Vamos analisar a colinearidade entre nossas variáveis numéricas. Para isso, vamos utilizar a matriz de correlação:

In [ ]:
sns.clustermap(tb_cov.corr(numeric_only=True), vmin=-1, vmax=1, cmap="Spectral")


A função `sns.clustermap()` nos permite visualizar grupos de variáveis correlatas (diretamente ou indiretamente). Na sua avaliação, temos grupos de variáveis correlatas? Como podemos tratar essa colinearidade sem descartar variáveis?

#### Distribuição

Como vimos quando trabalhamos no problema dos diamantes, variáveis positivas com distribuição muito assimétrica podem ser transformadas para melhorar o desempenho de alguns algoritmos. Vamos avaliar a distribuição de nossas variáveis numéricas utilizando histogramas.

In [ ]:
tb_melt = tb_cov.select_dtypes(include=np.number).melt()


In [ ]:
sns.displot(
    data=tb_melt,
    x="value",
    col="variable",
    kind="kde",
    col_wrap=3,
    facet_kws={"sharex": False, "sharey": False},
)


Poderíamos fazer a conversão de variáveis manualmente e inicializar nossos normalizadores, PCAs, Encoders separadamente. Mas o `sklearn` nos permite criar um pipeline de transformação de dados, que pode ser aplicado em qualquer conjunto de dados. Vamos criar um pipeline para transformar nossos dados e aplicá-lo em nosso dataset.

## Utilizando Pipelines

Pipelines são uma forma de automatizar o processo de transformação de dados. Eles são especialmente úteis quando temos muitas transformações a serem feitas, ou quando queremos aplicar a mesma transformação em vários conjuntos de dados diferentes (por exemplo, em um conjunto de treino e em um conjunto de teste).

In [ ]:
num_vars = [
    "Elevation",
    "Aspect",
    "Slope",
    "Horizontal_Distance_To_Hydrology",
    "Vertical_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways",
    "Hillshade_9am",
    "Hillshade_Noon",
    "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points",
]
cat_vars = ["Wilderness_Area", "Soil_Type"]

In [ ]:
X = tb_cov[num_vars + cat_vars]
y = le.transform(tb_cov["target"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


Agora, vamos desenhar o fluxo de transformações que serão executadas sobre nossas *features* de acordo com sua natureza (categórica ou contínua):

1. **Variáveis Contínuas**:
    1. Transformação Logarítmica (tratar assimetria);
    1. Normalização (etapa básica de pré-processamento);
    1. PCA (remover colinearidade)
1. **Variáveis Categóricas**:
    1. One-Hot Encoding (transformar em variáveis binárias dummies)

Vamos criar um pipeline para cada tipo de transformação e depois juntá-los em um pipeline único.

In [ ]:
from sklearn.pipeline import Pipeline

Um pipeline nada mais é que **uma sequência de transformações e modelos** que são executados em sequencia. Para criar um pipeline, precisamos de uma lista de tuplas, onde cada tupla contém o nome do passo e o objeto que será executado. Por exemplo, para criar um pipeline que transforma nossas variáveis contínuas, precisamos de uma lista de tuplas com os seguintes passos:

1. `PowerTransformer()`: transformação logarítmica;
1. `StandardScaler()`: normalização;
1. `PCA()`: redução de dimensionalidade e colinearidade.

Para isso, precisamos inicilizar cada um desses objetos:

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, PowerTransformer

pt = PowerTransformer()
sca = StandardScaler()
pca = PCA()

Agora, precisamos criar uma lista de tuplas com o nome de cada passo e o objeto que será executado:

In [ ]:
lista_etapas = [("POWER_TRANS", pt), ("STD_SCALER", sca), ("PCA", pca)]
num_pipeline = Pipeline(lista_etapas)

Pronto! Vamos textar nosso `pipeline`:

In [ ]:
num_pipeline.fit(X_train[num_vars])

In [ ]:
num_pipeline.transform(X_train[num_vars])


Perfeito! Agora vamos criar um pipeline para nossas variáveis categóricas:

In [ ]:
from sklearn.preprocessing import OneHotEncoder


ohe = OneHotEncoder()
cat_pipeline = Pipeline([("ONE_HOT", ohe)])


In [ ]:
cat_pipeline.fit(X_train[cat_vars])
cat_pipeline.transform(X_train[cat_vars])


Simples! Agora vamos juntar os dois pipelines em um único pipeline. Para isso, vamos utilizar o `ColumnTransformer` - um objeto que nos permite aplicar transformações diferentes em colunas diferentes. Esse objeto recebe como argumento uma lista de uplas onde cada upla tem o nome da etapa, o objeto que será executado e uma lista de nomes de colunas que serão transformadas por essa etapa:

In [ ]:
from sklearn.compose import ColumnTransformer

pipeline_preprocessamento = [
    ("NUMERICOS", num_pipeline, num_vars),
    ("CATEGORICOS", cat_pipeline, cat_vars),
]
data_prep_pipeline = ColumnTransformer(pipeline_preprocessamento)


In [ ]:
data_prep_pipeline.fit(X_train)


In [ ]:
data_prep_pipeline.transform(X_train)


É uma boa prática guardar todas as etapas de nosso pipeline em uma única célula, facilitando a adição, troca e remoção de etapas:

In [ ]:
# Pipeline Numérico
pt = PowerTransformer()
sca = StandardScaler()
pca = PCA()
lista_etapas = [("POWER_TRANS", pt), ("STD_SCALER", sca), ("PCA", pca)]
num_pipeline = Pipeline(lista_etapas)

# Pipeline Categórico
ohe = OneHotEncoder()
cat_pipeline = Pipeline([("ONE_HOT", ohe)])

# Pipeline de Preprocessamento
pipeline_preprocessamento = [
    ("NUMERICOS", num_pipeline, num_vars),
    ("CATEGORICOS", cat_pipeline, cat_vars),
]
data_prep_pipeline = ColumnTransformer(pipeline_preprocessamento)


Além de etapas de pré-processamento, podemos adicionar modelos ao nosso pipeline. Vamos adicionar um modelo de regressão logística ao nosso pipeline, o qual usaremos como baseline de erro:

In [ ]:
lm = LogisticRegression()
lm_pipeline = Pipeline([("PREP", data_prep_pipeline), ("LM", lm)])

Quando inserimos um modelo no final de nosso pipeline devemos incluir nossa variável target ao utilizar o método `.fit()`:

In [ ]:
lm_pipeline.fit(X_train, y_train)


Agora só precisamos avaliar o erro desse modelo...

## Erro Multi-Classe

Muitos das medidas de erro que vimos na última aula dependem do conceito de **Falso Positivo/Negativo** e **Verdadeiro Positivo/Negativo**. Esses conceitos são fáceis de entender quando temos apenas duas classes, mas como podemos aplicá-los em problemas com mais de duas classes?

In [ ]:
y_pred_lm = lm_pipeline.predict(X_test)


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

fig, ax = plt.subplots(1, 1, figsize=(15, 15))
cf = confusion_matrix(y_test, y_pred_lm, normalize="true")
disp = ConfusionMatrixDisplay(confusion_matrix=cf, display_labels=le.classes_)
disp.plot(ax=ax)

Vamos entender agora como podemos calcular o erro de um modelo de classificação multi-classe.

### Acurácia (do jeito errado)

Vamos começar calculando a **acurácia média**: o número de acertos totais dividido pelo número de exemplos totais:

In [ ]:
from sklearn.metrics import accuracy_score


print(f"Acurácia de {np.round(accuracy_score(y_test, y_pred_lm), 2)}")


O score acima tem um problema: ele não leva em consideração o desbalanceamento de classes, nesse exemplo agravado pela natureza multi-classe do problema. Perdemos toda visibilidade que nosso modelo, em algumas classes, tem quase 100% de erro!

Esse problema ocorrerá com **todas** as funções - elas serão resultados médios, que muitas vezes podem ocultar o que está acontecendo em nosso modelo.

### Classification Report

A função `classification_report()` nos dá uma visão mais detalhada do desempenho de nosso modelo. Ela nos dá a precisão, recall e f1-score para cada classe:

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_lm, target_names=le.classes_))


Embora a visão acima seja bem detalhada, ela sofre do mesmo problema que a matriz de confusão: não conseguimos comparar de forma direta o desempenho de cada modelo. Para isso precisaremos utilizar o F1-Score (e Precision/Recall).

Nesse caso teremos o mesmo problema que na acurácia: estaremos resumindo, de alguma forma, a performance de nosso modelo à um único número. Essa simplificação é feita, tradicionalmente, de duas formas:

1. **Weighted**: calcula o F1-Score para cada classe e tira a média, ponderando esta média pelo tamanho de cada classe.
2. **Macro**: calcula o F1-Score para cada classe e tira a média, sem ponderar esta média pelo tamanho de cada classe.

Podemos ver que essas duas formas de medir o erro são antagônicas: a primeira ignora o desbalanceamento de classes, enquanto a segunda dá importância igual à todas as classes. Qual delas devemos utilizar? Depende! Se prever **classes minoritárias é importante** (previsão de eventos raros, como fraudes ou incidência de doenças raras) devemos utilizar o **F1-Score Macro**. Se **não for crítico o desempenho em classes minoritárias**, por exemplo em um modelo de categorização da busca do cliente, podemos utilizar o **F1-Score Weighted**.

In [ ]:
f1_score(y_test, y_pred_lm, average="macro")


In [ ]:
print(f"Acurácia de {np.round(accuracy_score(y_test, y_pred_lm), 2)}")
print(f"F1-Score Macro de {np.round(f1_score(y_test, y_pred_lm, average='macro'), 2)}")
print(
    f"F1-Score Ponderado de {np.round(f1_score(y_test, y_pred_lm, average='weighted'), 2)}"
)

## Interpretando nossa Regressão

In [ ]:
tb_eval = X_train.copy()
probabilidades = lm_pipeline.predict_proba(X_train)
for i, tree in enumerate(le.classes_):
    tb_eval[f"prob_lm_{tree}"] = probabilidades[:,i]

In [ ]:
sns.heatmap(
    tb_eval.corr(numeric_only=True),
    vmin = -1, vmax = 1, center = 0, cmap = "Spectral"
)

In [ ]:
sns.scatterplot(
    data = tb_eval,
    x = "Elevation",
    y = "prob_lm_Spruce/Fir",
    s= 1, alpha = 0.1
)

Como nosso modelo tem muitas variáveis, embora exista uma relação clara entre a elevação e a probabilidade da cobertura ser de pinheiros-alemães, não conseguimos visualizar diretamente essa relação. Vamos construir uma base de dados específica para analisar o efeito marginal desta variável.

In [ ]:
tb_eval = pd.DataFrame()
tb_eval["Elevation"] = np.linspace(X_train["Elevation"].min(), X_train["Elevation"].max(), 1000)
for column in X_train.select_dtypes(include="number").columns:
    if column != "Elevation":
        tb_eval[column] = X_train[column].mean()
for column in X_train.select_dtypes(exclude="number").columns:
    tb_eval[column] = X_train[column].mode()[0]
tb_eval.head()

In [ ]:
probabilidades = lm_pipeline.predict_proba(tb_eval)
for i, tree in enumerate(le.classes_):
    tb_eval[f"prob_lm_{tree}"] = probabilidades[:,i]

In [ ]:
sns.lineplot(
    data = tb_eval,
    x = "Elevation",
    y = "prob_lm_Spruce/Fir"
)

# Redes Neurais

Redes neurais são uma classe de modelo preditivo que surgiu a partir da pesquisa sobre o funcionamento das células neuronais do cérebro - desenvolvida inicialmente por McCulloh e Pitts na década de 40. O modelo mais comum de rede neural, o **perceptron de múltiplas camadas (MLP)** foi desenvolvido em 1958 pelo psicológo Rosenblatt.

Os **MLPs** são caracterizados por dois atributos fundamentais:

* Número de neurônios por camada;
* Função de Ativação.

![MLP](images/multipercep.jpg)

Vamos criar um MLP para resolver nosso problema de classificação, variando o tamanho de nossa rede neural e analisando o resultado de nossas previsões:

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(4, 4), activation="relu", random_state=10)
mlp_pipeline = Pipeline([("PREP", data_prep_pipeline), ("MLP", mlp)])

In [ ]:
mlp_pipeline.fit(X_train, y_train)


Agora, vamos avaliar nosso modelo:

In [ ]:
y_pred_nn = mlp_pipeline.predict(X_test)
print(classification_report(y_test, y_pred_nn, target_names=le.classes_))

Vamos nos lembrar dos resultados da regressão:

In [ ]:
print(classification_report(y_test, y_pred_lm, target_names=le.classes_))

In [ ]:
probabilidades = mlp_pipeline.predict_proba(tb_eval)
for i, tree in enumerate(le.classes_):
    tb_eval[f"prob_nn_{tree}"] = probabilidades[:,i]
    
sns.lineplot(
    data = tb_eval,
    x = "Elevation",
    y = "prob_nn_Spruce/Fir",
    label = "MLP de (4, 4)"
)

sns.lineplot(
    data = tb_eval,
    x = "Elevation",
    y = "prob_lm_Spruce/Fir",
    label = "Regresão Logística"
)

Como podemos ver, pelo menos neste efeito, uma rede neural simples se comporta de maneira bem semelhante à nossa regressão. Vamos aumentar o número de neurônios em nossa rede neural e ver o que acontece:

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(30, 30, 30), activation="relu", random_state=10)
mlp_pipeline = Pipeline([("PREP", data_prep_pipeline), ("MLP", mlp)])
mlp_pipeline.fit(X_train, y_train)

In [ ]:
probabilidades = mlp_pipeline.predict_proba(tb_eval)
for i, tree in enumerate(le.classes_):
    tb_eval[f"prob_nn2_{tree}"] = probabilidades[:,i]
    
sns.lineplot(
    data = tb_eval,
    x = "Elevation",
    y = "prob_nn_Spruce/Fir",
    label = "MLP de (4, 4)"
)

sns.lineplot(
    data = tb_eval,
    x = "Elevation",
    y = "prob_nn2_Spruce/Fir",
    label = "MLP de (30, 30, 30)"
)

sns.lineplot(
    data = tb_eval,
    x = "Elevation",
    y = "prob_lm_Spruce/Fir",
    label = "Regresão Logística"
)

Claramente a rede complexa é muito menos linear que nossos dois outros modelos! Vamos avaliar o erro de nosso modelo:

In [ ]:
y_pred_nn2 = mlp_pipeline.predict(X_test)
print(classification_report(y_test, y_pred_nn2, target_names=le.classes_))

Uma melhoria significativa - especialmente nas classes minoritárias! Como todos os algoritmos de ML, redes neurais tem muitos hiperparâmetros que podem ser ajustados para melhorar o desempenho do modelo. Os principais são:

1. **Topologia da rede** - `hidden_layer_sizes`: número de camadas e número de neurônios por camada (em um MLP);
1. **Função de Ativação** - `activation`: função que determina o valor de saída de cada neurônio (hoje em dia quase nunca utilizamos funções diferentes da **ReLU**);
1. **Taxa de Aprendizagem** - `learning_rate_init`: o *learning rate* determina o quão agressiva a rede neural tenta se ajustar aos dados em cada iteração do algoritmo de otimização, padrão entre 0.00001 e 1;
1. **Número de Iterações** - `max_iter`: número de iterações do algoritmo de otimização (quantas vezes seu dataset completo passa pela rede).

Vamos ver como podemos alterar o `max_iter` buscando um ponto ótimo entre precisão e tempo de treinamento:

In [ ]:
f1_list = []
for i in range(5, 101, 10):
    mlp = MLPClassifier(hidden_layer_sizes=(30, 30, 30), activation="relu", random_state=10, max_iter = i)
    mlp_pipeline = Pipeline([("PREP", data_prep_pipeline), ("MLP", mlp)])
    mlp_pipeline.fit(X_train, y_train)
    y_pred_test = mlp_pipeline.predict(X_test)
    y_pred_train = mlp_pipeline.predict(X_train)
    f1_train = f1_score(y_train, y_pred_train, average="macro")
    f1_test = f1_score(y_test, y_pred_test, average="macro")
    f1_list.append([i, f1_train, f1_test])

In [ ]:
tb_f1 = pd.DataFrame(f1_list, columns = ["max_iter", "f1_train", "f1_test"])
sns.lineplot(data = tb_f1, x = "max_iter", y = "f1_train", label = "Treino")
sns.lineplot(data = tb_f1, x = "max_iter", y = "f1_test", label = "Teste")